In [2]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2
from YouTube import capstone

In [8]:
import google-api-python-client

SyntaxError: invalid syntax (<ipython-input-8-8d64b14cd922>, line 1)

In [ ]:
import google-auth-oauthlib google-auth-httplib2

# Using the Youtube API

In [6]:
!pip install --upgrade google-api-python-client

     |████████████████████████████████| 57 kB 308 kB/s eta 0:00:01
     |████████████████████████████████| 70 kB 657 kB/s eta 0:00:01
     |████████████████████████████████| 87 kB 756 kB/s  eta 0:00:01
     |███████████████████████████████ | 92 kB 25.7 MB/s eta 0:00:01     |████████████████████████████████| 95 kB 507 kB/s 
     |████████████████████████████████| 1.3 MB 9.0 MB/s eta 0:00:01     |████████████▎                   | 481 kB 9.0 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 5.4 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 114 kB/s  eta 0:00:01
  Created wheel for googleapis-common-protos: filename=googleapis_common_protos-1.51.0-py3-none-any.whl size=77592 sha256=3889f533bb560d59eba6331e98de92809cd428fbf0ccec1f70a9d9a31c627b15
  Stored in directory: /home/ty/.cache/pip/wheels/4c/a1/71/5e427276ceeff277fd76878d1b19fbf4587a2845015d86864b
Successfully built googleapis-common-protos


In [7]:
!pip install --upgrade google-auth-oauthlib google-auth-httplib2

Requirement already up-to-date: google-auth-httplib2 in /home/ty/anaconda3/lib/python3.7/site-packages (0.0.3)
     |████████████████████████████████| 147 kB 5.0 MB/s eta 0:00:01
